In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Lain\AppData\Local\Temp\ipykernel_17912\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [4]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [5]:
data[['text']]

,text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL..."
1,Will do.
2,Nora--Cheryl has emailed dozens of memos about...
3,Dear Sir=2FMadam=2C I know that this proposal ...
4,fyi
...,...
995,So what's the latest? It sounds contradictory ...
996,"TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO..."
997,Barb I will call to explain. Are you back in t...
998,Yang on travelNot free tonite.May work tomorrow


In [26]:
df_X = data[['text']]
df_y = data['label']

In [28]:
print(data)

                                                  text  label
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                             Will do.      0
2    Nora--Cheryl has emailed dozens of memos about...      0
3    Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                  fyi      0
..                                                 ...    ...
995  So what's the latest? It sounds contradictory ...      0
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...      1
997  Barb I will call to explain. Are you back in t...      0
998    Yang on travelNot free tonite.May work tomorrow      0
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...      0

[1000 rows x 2 columns]


In [27]:
print(df_X)
print(df_y)

                                                  text
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
..                                                 ...
995  So what's the latest? It sounds contradictory ...
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997  Barb I will call to explain. Are you back in t...
998    Yang on travelNot free tonite.May work tomorrow
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...

[1000 rows x 1 columns]
0      1
1      0
2      0
3      1
4      0
      ..
995    0
996    1
997    0
998    0
999    0
Name: label, Length: 1000, dtype: int64


In [7]:
# Your code
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['text']], data['label'], test_size=0.2, random_state=42)



## Data Preprocessing

In [8]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [29]:
import re

text_col = df_X['text']
# First we remove inline JavaScript/CSS, Then we remove html comments, Next we can remove the remaining tags
clean_text = text_col.str.replace(r'<(script|style).*?>*?</\1>', ' ',flags=re.DOTALL).str.replace(r'<!--.*?-->', ' ', flags=re.DOTALL).str.replace(r'<[^>]+>', ' ', flags=re.DOTALL)

df_X['text'] = clean_text

print(df_X)

                                                  text
0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
..                                                 ...
995  So what's the latest? It sounds contradictory ...
996  TRANSFER OF 36,759,000.00 MILLION POUNDS TO YO...
997  Barb I will call to explain. Are you back in t...
998    Yang on travelNot free tonite.May work tomorrow
999  sbwhoeopSunday February 21 2010 7:42 PMHShaunH...

[1000 rows x 1 columns]


C:\Users\Lain\AppData\Local\Temp\ipykernel_17912\194620957.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X['text'] = clean_text


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [30]:
# Your code
import re
def cleaning_text(text):
    #Remove all the special characters
    text = re.sub(r'[^\w\s]', ' ', text)

    #Remove numbers
    text = re.sub(r'\d+', ' ', text) 

    #Remove all single characters
    text = re.sub(r'\b\w\b', ' ', text)

    # Remove single characters from the start of words
    text = re.sub(r'\s\w\s', ' ', text)

    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove prefixed 'b' (if the text was byte-decoded like b'text')
    text = re.sub(r'^b\s+', '', text)

    text = text.lower()

    return text


df_X['text'] = df_X['text'].apply(cleaning_text)




C:\Users\Lain\AppData\Local\Temp\ipykernel_17912\3544385173.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X['text'] = df_X['text'].apply(cleaning_text)


In [31]:
print(df_X)

                                                  text
0    dear sir strictly private business proposal am...
1                                             will do 
2    nora cheryl has emailed dozens of memos about ...
3    dear sir fmadam know that this proposal might ...
4                                                  fyi
..                                                 ...
995  so what the latest it sounds contradictory and...
996  transfer of million pounds to youraccount my n...
997  barb will call to explain are you back in the ...
998    yang on travelnot free tonite may work tomorrow
999  sbwhoeopsunday february pmhshaunh just talked ...

[1000 rows x 1 columns]


## Now let's work on removing stopwords
Remove the stopwords.

In [32]:
# Your code
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [w for w in tokens if not w in stop_words]
    return " ".join(filtered_tokens)

df_X['text'] = df_X['text'].apply(remove_stopwords)



C:\Users\Lain\AppData\Local\Temp\ipykernel_17912\2823762146.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X['text'] = df_X['text'].apply(remove_stopwords)


In [33]:
df_X['text']

0      dear sir strictly private business proposal mi...
1                                                       
2      nora cheryl emailed dozens memos haiti weekend...
3      dear sir fmadam know proposal might surprise e...
4                                                    fyi
                             ...                        
995    latest sounds contradictory af decide shall ta...
996    transfer million pounds youraccount name mr ej...
997                       barb call explain back country
998         yang travelnot free tonite may work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Name: text, Length: 1000, dtype: object

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [34]:
# Your code

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def apply_lemm(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(lemmatized_tokens)

df_X['text'] = df_X['text'].apply(apply_lemm)



C:\Users\Lain\AppData\Local\Temp\ipykernel_17912\729534871.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_X['text'] = df_X['text'].apply(apply_lemm)


In [35]:
print(df_X[['text']])

                                                  text
0    dear sir strictly private business proposal mi...
1                                                     
2    nora cheryl emailed dozen memo haiti weekend p...
3    dear sir fmadam know proposal might surprise e...
4                                                  fyi
..                                                 ...
995  latest sound contradictory af decide shall tak...
996  transfer million pound youraccount name mr eji...
997                     barb call explain back country
998       yang travelnot free tonite may work tomorrow
999  sbwhoeopsunday february pmhshaunh talked shaun...

[1000 rows x 1 columns]


In [36]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [39]:
print(X_train)

                                                  text
29   regard mr nelson smith kindly reply private em...
535         able reach oscar supposed send pdb receive
695  huma abedin checking pat work jack jake rest a...
557                             announced monday today
836  bank africaagence san pedro bp san pedro cote ...
..                                                 ...
106  adama ibrahim_________________________________...
270                                      mean schedule
860  dear friend compliment guess letter may come s...
435  dear president fdirector name mr micheal ipenz...
102  let know today tomorrow work would rather find...

[800 rows x 1 columns]


In [40]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train['text']).toarray()

print("Bag of Words:\n", X)
print("Vocabulary:", vectorizer.get_feature_names_out())


Bag of Words:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Vocabulary: ['___________________' '_____________________'
 '__________________________' ... 'â½trangers' 'â½x' 'â½ã']


In [41]:
df = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
df.head()

,___________________,_____________________,__________________________,_____________________________,__________________________________,_______________________________________,______________________________________________________,_________________________________________________________,_________________________________________________________________________,___________________________________________________________________________,...,â½ro,â½s,â½t,â½ta,â½te,â½tica,â½to,â½trangers,â½x,â½ã
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3,random_state=42)
kmeans.fit(X)
pred = kmeans.predict(X)

c:\Users\Lain\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [50]:
predict_df = pd.concat([df_X['text'], data['label'],pd.DataFrame(pred,columns=['class'])],axis=1)
predict_df.head()

,text,label,class
0,dear sir strictly private business proposal mi...,1,1.0
1,,0,1.0
2,nora cheryl emailed dozen memo haiti weekend p...,0,1.0
3,dear sir fmadam know proposal might surprise e...,1,1.0
4,fyi,0,1.0


## Extra features

In [18]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

NameError: name 'data_train' is not defined

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code